In [1]:
import cv2
import sys
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential #Initialise our neural network model as a sequential network
from keras.layers import Conv2D #Convolution operation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation#Applies activation function
from keras.layers import Dropout#Prevents overfitting by randomly converting few outputs to zero
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector
from keras.layers import Dense # Regular fully connected neural network
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import model_from_json

Using TensorFlow backend.


In [3]:
image = 'test_image1.png'
cascPath = "haarcascade_frontalface_default.xml"

In [ ]:
imagePath = 'test_image1.png'
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
img = cv2.imread(imagePath)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(
gray,
scaleFactor=1.1,
minNeighbors=5,
minSize=(30, 30),
flags = cv2.CASCADE_SCALE_IMAGE)
print("Found {0} faces!".format(len(faces)))
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2.imshow("Faces found", img)
cv2.waitKey(0)
    

In [4]:
json_file = open('fer.json','r')
loaded_model_json = json_file.read()
json_file.close()

In [5]:
loaded_model = model_from_json(loaded_model_json)

In [6]:
loaded_model.load_weights("fer.h5")

In [7]:
gender_model = load_model('pre-trained/gender_detection.model')

In [8]:
WIDTH = 48
HEIGHT = 48
x=None
y=None
labels_emo = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
labels_gen = ['Man','Woman']

In [9]:
img_rgb = cv2.imread(image)
img_gray=cv2.cvtColor(img_rgb,cv2.COLOR_RGB2GRAY)

In [10]:
faceCascade = cv2.CascadeClassifier(cascPath)
faces = faceCascade.detectMultiScale(
        img_gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE
)

In [11]:
emotion=[]
for (x, y, w, h) in faces:
        roi_gray = img_gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #predicting the emotion
        yhat= loaded_model.predict(cropped_img)
        cv2.putText(img_rgb, labels_emo[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        print("Emotion: "+labels_emo[int(np.argmax(yhat))])
        emotion.append(labels_emo[int(np.argmax(yhat))])
        

Emotion: Happy
Emotion: Happy
Emotion: Surprise
Emotion: Fear


In [12]:
gender=[]
for (x, y, w, h) in faces:
        roi_rgb = img_rgb[y:y + h, x:x + w]
        cropped_img = cv2.resize(roi_rgb, (96,96))
        cropped_img = cropped_img.astype("float") / 255.0
        cropped_img = img_to_array(cropped_img)
        cropped_img = np.expand_dims(cropped_img, axis=0)
        #predicting the gender
        ghat= gender_model.predict(cropped_img)
        cv2.putText(img_rgb, labels_gen[int(np.argmax(ghat))], (x, y+100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        print("Gender: "+labels_gen[int(np.argmax(ghat))])
        gender.append(labels_gen[int(np.argmax(ghat))])
        

Gender: Man
Gender: Man
Gender: Woman
Gender: Man


In [ ]:
cv2.imshow('Emotion', img_rgb)
cv2.waitKey()

In [14]:
myfile = open('data.txt','w')

In [15]:
myfile.write(str(len(faces)))
myfile.write("\r\n")
myfile.write(str(emotion))
myfile.write("\r\n")
myfile.write(str(gender))
myfile.write("\r\n")
print('Data Saved to the file')

Data Saved to the file


In [21]:
import pandas as pd
d = {'Image':image, 'Number of faces':len(faces), 'Emotion':[emotion],'Gender':[gender]}
df = pd.DataFrame(data=d)

In [22]:
df

,Emotion,Gender,Image,Number of faces
0,"[Happy, Happy, Surprise, Fear]","[Man, Man, Woman, Man]",test_image1.png,4


In [23]:
df.to_excel("output.xlsx")  